# Laboratório Elastic Search

## Contexto
O objetivo deste laboratório é explorar diferentes mecanismos de busca dentro e fora do Elastic Search!
Para isto, iremos explorar uma base de verificações de notícia de agência de checagem chamada Lupa.
O papel de uma agência de checagem é analisar uma notícia e verificar sua veracidade, gerar um veredito e retornar o veredito da sua análise.

Neste laboratório, iremos nos preocupar apenas com o texto da análise e em como recuperá-los através de diferentes estratégias de busca.

## Tarefa
- Cada uma das equipes receberá 3 queries e deverá montar mais 3 queries (pergunta ou declaração textual para realizar a busca). As equipes podem se inspirar nas notícias fornecidas (.csv) para montá-las.
- Com as 6 queries em mãos, cada grupo irá realizar 4 tipos de busca. Uma busca léxica com BM25 (pelo ElasticSearch), uma busca semântica (pelo ElasticSearch), uma busca híbrida (manualmente utilizando as duas buscas anteriores) e uma estratégia de busca de sua preferência (busca criativa) diferente das anteriores.
    - As buscas possuem métodos de pré-processamento, sintam-se livres para testar diferentes combinações dos pré-processamentos em quaisquer uma das buscas!
    - Inclusive as equipes podem criar outro métodos de pré-processamento.
    - A busca criativa pode ser inspirada na busca léxica, semântica, híbrida ou algo totalmente novo, o importante é que ela traga bons resultados para na competição!
- Para cada uma das 4 buscas, os grupos devem coletar os top 10 resultados para as 6 queries, resultando em 240 resultados. Diferentes estratégias de busca tendem a ter intersecção entre si nos resultados, então o número de resultados únicos será bem menor que 240 (não se preocupem).
- A próxima etapa é de anotação, os grupos devem anotar o quão bom cada resultado é em relação com à query.
    - A rotulação deve possui uma gradação em 3 níveis:
        - 0: Não é Relevante
        - 1: Pouco Relevante
        - 2: Muito Relevante
    - Os grupos só precisam anotar os resultados únicos agrupados por query, ou seja, se um mesmo resultado for encontrado na busca léxica da query A e na busca semântica da query A, ele só precisa ser anotado uma única vez (pois anotação deste resultado é igual para ambas as buscas). O código abaixo já realiza esta otimização no processo de rotulagem.
Entretanto a anotação de um mesmo resultado na busca léxica da query A pode ser diferente da anotação na busca semântica da query B.
    - Os resultados desta rotulação serão importantes para que o seu grupo saiba o quão boa sua busca é e como ela se compara com as buscas "tradicionais".

## Artefatos entregáveis
- As 3 queries definidas pela equipe.
- A implementação da busca criativa, bem com seus imports, arquivos de dependências (requirements.txt), configurações etc.
- Todos os resultados da busca.
- Este notebook atualizado e funcionando com o botão "Rodar tudo".

## Competição
A competição será realizada através da comparação da sua implementação da busca criativa com uma base interna.
Para que você atinja um bom desempenho na competição é importante se atentar ao seu processo de criação de queries e de rotulação, pois ele será o guia em o quão boa sua busca está se saindo!

Esta competição irá utilizar o NDCG score e média dos scores para verificar qual grupo obteve a melhor ordenação média dos resultados e melhor obtenção de resultados.

## Ferramental
Para executar este lab não é necessário uma máquina com GPU, mas a máquina deve ter capacidade de virtualização, além de possui o Docker e o python instalado.

Será utilizados o ElasticSearch (e opcionalmente o Kibana) no ambiente docker.

É recomendado o uso de Python 3.12 para realizar o lab, além de um venv ou ambiente conda.
Você deve instalar as dependências do requirements.txt.

## Detalhes
Assim que o grupo executar suas buscas será gerada a primeira planilha (search_results_for_labeling.xlsx), o grupo deve rotulá-la e em seguida rodar a avaliação do código (NDCG e média).
Após a rotulação ser finalizada, execute o código que gera a "search_results.xslx" para visualizar os rótulos e a ordem dos rótulos de modo mais mastigado.

Ao executar o docker compose além de ser levantado o ElasticSearch, também é levantada uma interface visual chamada Kibana.

O Kibana é um frontend para facilitar o acionamento de algumas operações do ElasticSearch e controle de configurações específicas, observar métricas etc. Ela pode ser utilizada para fins de debug para quem tiver curiosidade acesso o URI: http://localhost:5601 após levantar o serviço com o docker compose.

## Preparação do ambiente

### Imports

In [ ]:
import zipfile
import requests
import os
import pandas as pd
from datetime import datetime
import re
import warnings
import subprocess
warnings.filterwarnings("ignore")
from collections import OrderedDict

from elasticsearch import Elasticsearch

from sentence_transformers import SentenceTransformer

import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer
import sklearn

nltk.download('punkt_tab')
nltk.download('stopwords')

import spacy
import unidecode

### Passo 0: Subir Stack do Elastic (ElasticSearch e Kibana)

In [ ]:
!python -m spacy download pt_core_news_sm
# Caso o comando acima não funcione, execute-o no terminal

In [ ]:
subprocess.call(["docker", "compose", "up", "-d"]) # Se esse comando falhar ou retornar 1, você pode executar ele diretamente no terminal para identificar o erro

### Passo 1: Baixar dados do Lupa

In [5]:
url = "https://docs.google.com/uc?export=download&confirm=t&id=1W067Md2EbvVzW1ufzFg17Hf7Y9cCZxxr"
filename = "articles_lupa_lab_elasticsearch.zip"
data_path = "data"
zip_file_path = f"{data_path}/{filename}"

os.makedirs(data_path, exist_ok=True)

# Download file
with open(zip_file_path, "wb") as f:
    f.write(requests.get(url, allow_redirects=True).content)

# Extract file    
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(data_path)
    
output_file = f"{data_path}/articles_lupa.csv"
assert os.path.exists(output_file)

### Opções de pré-processamento

In [6]:
class Preprocessors:
    STOPWORDS = set(nltk.corpus.stopwords.words('portuguese'))
    
    def __init__(self):
        self.stemmer = PorterStemmer()
        self.spacy_nlp = spacy.load("pt_core_news_sm")
        # self.lemma = WordNetLemmatizer()
        
    
    def remove_stopwords(self, text):
        """
        Return :- String without stopwords
        Input :- String
        Output :- String
        """
        # word tokenization
        tokens = word_tokenize(text)
        # remove stopwords from tokens
        tokens = [word for word in tokens if word not in self.STOPWORDS]
        # join list with space separator as string
        return ' '.join(tokens)
    
    def lemma(self, text):
        return " ".join([token.lemma_ for token in self.spacy_nlp(text)])
    
    def porter_stemmer(self, text):
        # word tokenization
        tokens = word_tokenize(text)

        for index in range(len(tokens)):
            # stem word to each word
            stem_word = self.stemmer.stem(tokens[index])
            # update tokens list with stem word
            tokens[index] = stem_word

        # join list with space separator as string
        return ' '.join(tokens)

    def lower_case(self, str):
        return str.lower()

    def remove_urls(self, text):
        url_pattern = r'https?://\S+|www\.\S+'
        without_urls = re.sub(pattern=url_pattern, repl=' ', string=text)
        return without_urls

    def remove_numbers(self, text):
        number_pattern = r'\d+'
        without_number = re.sub(pattern=number_pattern,
    repl=" ", string=text)
        return without_number

    def accented_to_ascii(self, text):
        # apply unidecode function on text to convert
        # accented characters to ASCII values
        text = unidecode.unidecode(text)
        return text

### Passo 2: Pré-processar os dados e gerar embeddings

In [ ]:
# Carregar o modelo de embeddings
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

data_df_path = "data/data_df.pkl"

# Selecione diferentes pré-processamentos se tiver curiosidade
preprocessor = Preprocessors()
    
preprocessing_steps = [
    preprocessor.lower_case,
    preprocessor.remove_urls,
    preprocessor.remove_numbers,
    preprocessor.remove_stopwords,
    preprocessor.lemma,
    preprocessor.accented_to_ascii
]

if not os.path.exists(data_df_path):    
    df = pd.read_csv(output_file, sep=";")[["Título", "Texto", "Data de Publicação"]]
    df["Data de Publicação"] = df["Data de Publicação"].apply(lambda str_date: datetime.strptime(str_date.split(" - ")[0], "%d.%m.%Y"))
    df.sort_values("Data de Publicação", inplace=True, ascending=False)
    df["Embeddings"] = [None] * len(df)
    df["id"] = df.reset_index(drop=True).index

    for i, row in df.iterrows():
        texto_completo = row["Texto"].strip() + "\n" + row["Título"].strip()
        
        df.at[i, "Texto completo"] = texto_completo
        texto_processado = texto_completo
        for preprocessing_step in preprocessing_steps:
            texto_processado = preprocessing_step(texto_processado)
        
        df.at[i, "Texto processado"] = texto_processado
        embeddings = model.encode(texto_completo).tolist()
        df.at[i, "Embeddings"] = embeddings
        
    print("Geração de embeddings finalizada.")
    
    with open(data_df_path, "wb") as f:
        df.to_pickle(f)
else:
    with open(data_df_path, "rb") as f:
        df = pd.read_pickle(f)
    print("Dataframe carregado de arquivo.")

### Passo 3: Indexar dados no ElasticSearch

In [8]:
es = Elasticsearch(
    hosts = [{'host': "localhost", 'port': 9200, "scheme": "https"}],
    basic_auth=("elastic","elastic"),
    verify_certs = False,
)

In [ ]:
RECREATE_INDEX = True

index_name = "verificacoes_lupa"

# Se a flag for True e se ele existir, deleta o índice
if RECREATE_INDEX and es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"Índice '{index_name}' deletado.")

# Cria o índice e popula com os dados
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, mappings={
        "properties": {
            "id": {"type": "integer"},
            "full_text": {"type": "text"},
            "processed_text": {"type": "text"},
            "embeddings": {"type": "dense_vector", "dims": 384}
        }
    })
    print(f"Índice '{index_name}' criado.")
    
    for i, row in df.iterrows():
        es.index(index=index_name, id=row["id"], body={
            "id": row["id"],
            "full_text": row["Texto completo"],
            "processed_text": row["Texto processado"],
            "embeddings": row["Embeddings"]
        })
    print("Índice preenchido.")

print("Indexação finalizada.")

## Tarefas

### Tarefa 1: Criar query
Preen

Inspire-se nas notícias presente no csv (articles_lupa.csv) para gerar uma query interessante.

### Tarefa 2: Executar buscas com todas as queries
Agora com as queries obtidas de todos os grupos, realize cada uma das buscas para cada uma das queries.

In [ ]:
with open("data/queries_fixadas.txt", "r") as f:
    queries_fixadas = [line.strip() for line in f.readlines()]
    
# Preencha aqui as queries do grupo
QUERY_1 = XXXXXX
QUERY_2 = XXXXXX
QUERY_3 = XXXXXX
    
queries_do_grupo = [QUERY_1, QUERY_2, QUERY_3]

queries = [*queries_fixadas, *queries_do_grupo]

assert len(queries) == 6

all_results = {}

#### Busca Léxica

In [11]:
def lexical_search(queries):
    lexical_results = {}
    for query in queries:
        # Pré-processa os dados
        for preprocessing_step in preprocessing_steps:
            query = preprocessing_step(query)
        
        search_query = {
            "query": {
                "match": {
                    "processed_text": query
                }
            }
        }

        response = es.search(index=index_name, body=search_query)

        hits_results = []
        for hit in response["hits"]["hits"][:10]:
            hits_results.append((hit["_source"]["id"], hit["_score"]))
        lexical_results[query] = hits_results
        
    return lexical_results

all_results["lexical"] = lexical_search(queries)

#### Busca Semântica

In [ ]:
def semantic_search(queries):
    semantic_results = {}
    
    for query in queries:
        # Pré-processa os dados
        for preprocessing_step in preprocessing_steps:
            query = preprocessing_step(query)
            
        query_vector = model.encode(query).tolist()
        
        search_query = {
            "query": {
                "script_score": {
                    "query": {"match_all": {}},
                    "script": {
                        "source": "cosineSimilarity(params.query_vector, 'embeddings') + 1.0",
                        "params": {"query_vector": query_vector}
                    }
                }
            }
        }

        response = es.search(index=index_name, body=search_query)
        hits_results = []
        for hit in response["hits"]["hits"][:10]:
            hits_results.append((hit["_source"]["id"], hit["_score"]))
            # print(hit["_source"], "Score:", hit["_score"])
        semantic_results[query] = hits_results

    return semantic_results

    # Perform vector similarity search

all_results["semantic"] = semantic_search(queries)

#### Busca Híbrida

In [13]:
def hybrid_search(queries):
    ## TODO: Implementar busca híbrida
    pass

all_results["hybrid"] = hybrid_search(queries)

#### Busca Criativa

In [14]:
def creative_search(queries):
    ## TODO: Implementar busca híbrida
    pass

all_results["creative"] = creative_search(queries)

### Tarefa 3: Rotular dados da planilha para avaliar métodos de busca

In [ ]:
search_results_df = pd.DataFrame(all_results)
search_results_df

In [16]:
# Função auxiliar que mapea o doc id para o texto correspondente da notícia
def get_mapping_doc_id_to_text(data):
    mapping_key_to_text = {}
    for search_type in data.keys():
        for query in data[search_type].keys():
            for result in data[search_type][query]:
                mapping_key_to_text[result[0]] = df.iloc[result[0]]["Texto completo"]
    return mapping_key_to_text

In [ ]:
len(all_results["lexical"])

In [ ]:
all_results["lexical"].keys()

In [ ]:
# Função que estrutura os dados para serem salvos em um arquivo Excel para rotulação
def structure_data_to_excel_for_labeling(data, map_doc_id_to_text, query_to_idx, filename="search_results_for_labeling.xlsx"):
    with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
        for query, query_idx_str in query_to_idx.items():
            rows = []
            
            lexical_results = data.get("lexical", {}).get(query, [])
            semantic_results = data.get("semantic", {}).get(query, [])
            hybrid_results = data.get("hybrid", {}).get(query, [])
            creative_results = data.get("creative", {}).get(query, [])
            
            all_results = [*lexical_results, *semantic_results, *hybrid_results, *creative_results]
            
            document_ids = sorted(list({document_id for (document_id, _) in all_results}))
            
            max_len = max(len(lexical_results), len(semantic_results), len(hybrid_results), len(creative_results))
            lexical_results += [("", "")] * (max_len - len(lexical_results))
            semantic_results += [("", "")] * (max_len - len(semantic_results))
            hybrid_results += [("", "")] * (max_len - len(hybrid_results))
            creative_results += [("", "")] * (max_len - len(creative_results))
            
            for document_id in document_ids:
                rows.append([document_id, map_doc_id_to_text[document_id], ""])
            
            df = pd.DataFrame(rows, columns=["ID", "Resultados", "Avaliação"])
            
            workbook = writer.book
            # worksheet = workbook.add_worksheet(query[:30])
            worksheet = workbook.add_worksheet(query_idx_str)
            writer.sheets[query] = worksheet
            
            # Format for big header
            merge_format = workbook.add_format({
                'bold': True,
                'align': 'center',
                'valign': 'vcenter',
                'font_size': 14,
                'bg_color': '#D9E1F2',
                'border': 1
            })
            worksheet.merge_range(0, 0, 0, 6, f"Query: {query}", merge_format)
            
            # Format for column headers
            header_format = workbook.add_format({
                'bold': True,
                'bg_color': '#B4C6E7',
                'border': 1,
                'align': 'center',
                'valign': 'vcenter'
            })
            
            # Write column headers
            worksheet.write(1, 0, "ID", header_format)
            worksheet.merge_range(1, 1, 1, 5, "Resultados", header_format)
            worksheet.write(1, 6, "Avaliação", header_format)
                
            # Format for data cells
            data_format = workbook.add_format({'border': 1})
            
            for row_num, row_data in enumerate(df.values):
                # print(text)
                worksheet.write(row_num + 2, 0, row_data[0], data_format)
                worksheet.merge_range(row_num + 2, 1, row_num + 2, 5, row_data[1], data_format)  # Merge B:F for "Review"
                
            
            # Add dropdown to "Review" columns
            dropdown_format = workbook.add_format({'border': 1, 'bg_color': '#E2EFDA'})
            for col in [6]:
                dropdown_range = f"{chr(65 + col)}3:{chr(65 + col)}{len(df) + 2}"
                worksheet.data_validation(dropdown_range, {
                    'validate': 'list',
                    'source': ['0', '1', '2'],
                    'input_message': 'Select a review option (0, 1, 2)'
                })
                for row_num in range(2, len(df) + 2):
                    worksheet.write(row_num, col, "", dropdown_format)
            
            # Adjust column width
            for i, column in enumerate(df.columns):
                worksheet.set_column(i, i, max(15, len(column) + 2))
    
    print(f"Data written to {filename}")

all_queries = []
for search_type in all_results:
    if not all_queries:
        all_queries = sorted(list(all_results[search_type].keys()))
    assert all_queries == sorted(list(all_results[search_type].keys()))

query_to_idx = OrderedDict()
for idx, query in enumerate(all_queries):
    query_to_idx[query] = f"query_{idx}"

# Convert and save to Excel
labeling_filename = "search_results_for_labeling.xlsx"
structure_data_to_excel_for_labeling(all_results, get_mapping_doc_id_to_text(all_results), query_to_idx, labeling_filename)

# Rotule o arquivo search_results_for_labeling.xlsx

### Tarefa 4: Após rotulado o arquivo execute o código abaixo para computar o NDCG score dos diferentes métodos de busca

In [45]:
# Carrega o arquivo rotulado
all_sheets_labels = pd.read_excel(labeling_filename, sheet_name=None, skiprows=1)
labeling_df = {query_key: all_sheets_labels[query_key][["ID", "Avaliação"]].set_index("ID") for query_key in all_sheets_labels.keys()}

In [ ]:
# Função que salva os resultados obtido no Excel para uma melhor visualização
def structure_data_to_excel(data, labeling_df, query_to_idx, filename="search_results.xlsx"):
    with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
        for query, query_idx_str in query_to_idx.items():
            rows = []
            
            lexical_results = data.get("lexical", {}).get(query, [])
            semantic_results = data.get("semantic", {}).get(query, [])
            hybrid_results = data.get("hybrid", {}).get(query, [])
            creative_results = data.get("creative", {}).get(query, [])
            
            max_len = max(len(lexical_results), len(semantic_results))
            lexical_results += [("", "")] * (max_len - len(lexical_results))
            semantic_results += [("", "")] * (max_len - len(semantic_results))
            hybrid_results += [("", "")] * (max_len - len(hybrid_results))
            creative_results += [("", "")] * (max_len - len(creative_results))
            
            for (lex_res, lex_score), (sem_res, sem_score), (hyb_res, hyb_score), (cre_res, cre_score) in zip(lexical_results, semantic_results, hybrid_results, creative_results):
                cur_labeling_df = labeling_df[query_idx_str]

                rows.append([
                    lex_res, lex_score, cur_labeling_df.loc[lex_res]["Avaliação"],
                    sem_res, sem_score, cur_labeling_df.loc[sem_res]["Avaliação"],
                    hyb_res, hyb_score, cur_labeling_df.loc[hyb_res]["Avaliação"],
                    cre_res, cre_score, cur_labeling_df.loc[cre_res]["Avaliação"]]
                )
            
            df = pd.DataFrame(rows, columns=["Lexical Result", "Lexical Score", "Review", "Semantic Result", "Semantic Score", "Review", "Hybrid Result", "Hybrid Score", "Review", "Creative Result", "Creative Score", "Review"])
            workbook = writer.book
            # worksheet = workbook.add_worksheet(query[:30])
            worksheet = workbook.add_worksheet(query_idx_str)
            
            
            # writer.sheets[query] = worksheet
            
            # Format for big header
            merge_format = workbook.add_format({
                'bold': True,
                'align': 'center',
                'valign': 'vcenter',
                'font_size': 14,
                'bg_color': '#D9E1F2',
                'border': 1
            })
            worksheet.merge_range(0, 0, 0, 5, f"Query: {query}", merge_format)
            
            # Format for column headers
            header_format = workbook.add_format({
                'bold': True,
                'bg_color': '#B4C6E7',
                'border': 1,
                'align': 'center',
                'valign': 'vcenter'
            })
            
            # Write column headers
            for col_num, column_name in enumerate(df.columns):
                worksheet.write(1, col_num, column_name, header_format)
            
            # Format for data cells
            data_format = workbook.add_format({'border': 1})
            
            # Write data
            for row_num, row_data in enumerate(df.values):
                for idx, inner_data in enumerate(row_data):
                    worksheet.write(row_num + 2, idx, inner_data, data_format)
                    
            # Adjust column width
            for i, column in enumerate(df.columns):
                worksheet.set_column(i, i, max(15, len(column) + 2))
        
    print(f"Data written to {filename}")

# Convert and save to Excel
structure_data_to_excel(all_results, labeling_df, query_to_idx)

#### Avaliação da ordem

In [ ]:
def compute_all_ndcg(data, query_to_idx, labeling_df):
    ndcg_scores = {}
    
    for search_type in data.keys():
        all_labels = {}
        for query, query_idx_str in query_to_idx.items():
            search_results = data[search_type][query]
            scores = [int(labeling_df[query_idx_str].loc[doc_id]["Avaliação"]) for doc_id, _ in search_results]
            all_labels[query] = sklearn.metrics.ndcg_score([scores], [sorted(scores,reverse=True)])
        ndcg_scores[search_type] = all_labels

    return ndcg_scores

# Computa o score NDCG@10
def compute_mean_ndcg(data, query_to_idx, labeling_df):
    ndcg_scores = {}
    
    for search_type in data.keys():
        all_labels = []
        for query, query_idx_str in query_to_idx.items():
            search_results = data[search_type][query]
            scores = [int(labeling_df[query_idx_str].loc[doc_id]["Avaliação"]) for doc_id, _ in search_results]
            all_labels.append(scores)
        ndcg_scores[search_type] = sklearn.metrics.ndcg_score(all_labels, [sorted(score_group,reverse=True) for score_group in all_labels])
    
    return ndcg_scores

mean_ndcg_results = compute_mean_ndcg(all_results, query_to_idx, labeling_df)
all_ndcg_results = compute_all_ndcg(all_results, query_to_idx, labeling_df)

search_keys = ["lexical", "semantic", "hybrid", "creative"]
print("NDCG@10 Médio")
for key in search_keys:
    print(f"\tBusca {key}: {(100 * mean_ndcg_results[key]):.2f}%")
print("\n----------------\n")
print("NDCG@10 para cada query")
for key in search_keys:
    print(f"{key}:")
    for query, ndcg_score in all_ndcg_results[key].items():
        print(f"\tQuery '{query}': {(100 * ndcg_score):.2f}%")

#### Avaliação da qualidade

In [ ]:
def compute_mean(data, query_to_idx, labeling_df):
    ndcg_scores = {}
    
    for search_type in data.keys():
        all_labels = {}
        for query, query_idx_str in query_to_idx.items():
            search_results = data[search_type][query]
            scores = [int(labeling_df[query_idx_str].loc[doc_id]["Avaliação"]) for doc_id, _ in search_results]
            all_labels[query] = sum(scores)/(sum([2] * len(scores)))
        ndcg_scores[search_type] = all_labels
    
    return ndcg_scores

def compute_mean_average(data, query_to_idx, labeling_df):
    mean_average_scores = {}
    
    for search_type in data.keys():
        all_labels = []
        for query, query_idx_str in query_to_idx.items():
            search_results = data[search_type][query]
            scores = [int(labeling_df[query_idx_str].loc[doc_id]["Avaliação"]) for doc_id, _ in search_results]
            all_labels.append(sum(scores)/(sum([2] * len(scores))))
        mean_average_scores[search_type] = sum(all_labels)/len(all_labels)
    
    return mean_average_scores

mean_average_results = compute_mean_average(all_results, query_to_idx, labeling_df)
# all_ndcg_results = compute_mean(all_results, query_to_idx, labeling_df)

search_keys = ["lexical", "semantic", "hybrid", "creative"]
print("Média por busca")
for key in search_keys:
    print(f"\tBusca {key}: {(100 * mean_average_results[key]):.2f}%")
print("\n----------------\n")
print("Média por query e por busca")
for key in search_keys:
    print(f"{key}:")
    for query, ndcg_score in all_ndcg_results[key].items():
        print(f"\tQuery '{query}': {(100 * ndcg_score):.2f}%")